The following code is the continuation of Section 4.1 of the Text Summarization_Main.R script.

4.1.1 Obtaining Doc2Vec embeddings



In [32]:
# Import modules
import pandas as pd
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import gensim
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
import unihandecode
import string
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('english'))
stemmer= PorterStemmer()
import itertools as it
from gensim.models.doc2vec import Doc2Vec
import collections

In [4]:
# Set a pattern for the file names to load:
file_name = r'C:\Users\Aigerim\Documents\R\Thesis\transcripts_df\transcript{}.csv'

# Create an empty dataframe to store all the sentences:
columns = ['doc_id','text'] # Columns in the data frame
df = pd.DataFrame(columns=columns)
df = df.fillna(0) 

# Import the data frames with sentences into a single data frame:
for k in range(1,21):    
    # Read a data frame corresponding to a transcript (features: sentence ids and sentences):
    df1 = pd.read_csv(file_name.format(k), encoding='latin-1')
    df1 = df1.drop(columns=["Unnamed: 0"])
    df = df.append(df1)
df.to_csv(r'~/Documents/R/Thesis/df.csv')

,doc_id,text
0,2,We are delighted that you are able to be with ...
1,3,"The presentation will, as usual, follow our cu..."
2,4,"Firstly, we will begin with an overview of the..."
3,5,We'd also like to point out that we are only t...
4,6,So please ask your question only through our w...
...,...,...
333,334,The rest of these items are all consistent wit...
334,335,"DD&A, we did lower that guidance range, really..."
335,336,"And then interest expense is down, reflecting ..."
336,337,And you'll see the next step down on interest ...


In [22]:
# Clean and pre-process the sentences:
sentences_df = df
nrow = sentences_df.shape[0] # Extract the total number of sentences
for r in range(0, nrow):
    sentences_df.iloc[r,1] = unihandecode.unidecode(sentences_df.iloc[r,1]) # convert to ASCII 
    sentences_df.iloc[r,1] = re.sub(r'[^\w\s\"!\?]',' ', sentences_df.iloc[r,1]) # Remove punctuation except ! ? "
    sentences_df.iloc[r,1] = re.sub(r"\d+", " 123", sentences_df.iloc[r,1]) # Replace numbers with '123'
    sentences_df.iloc[r,1] = sentences_df.iloc[r,1].strip() # Remove redundant white space
    word_tokens = word_tokenize(sentences_df.iloc[r,1]) # Tokenize each sentence on word level
    stemmed = [stemmer.stem(word) for word in word_tokens] # Stemming using Porter's algorithm
    sentences_df.iloc[r,1] = ' '.join(stemmed) # Paste the tokens into one string (for each sentence)
    
# Create a Tagged document with each sentence tagged (an object needed as an input to the Doc2Vec model):
tagged_data = [TaggedDocument(words=nltk.word_tokenize(_d.lower()), 
                                  tags=[str(i)]) for i, _d in enumerate(sentences_df["text"])]

In [40]:
# Hyperparameters tuning: setting values for embedding vector size and context window size:
vec_size = [25, 50, 100] # Embedding vector size variations used for tuning
window_size = [2,3,4] # Window size variations used for tuning

# Expand the grid:
a = [i for i in it.product(vec_size,window_size)] # 9 combinations
a

[(25, 2),
 (25, 3),
 (25, 4),
 (50, 2),
 (50, 3),
 (50, 4),
 (100, 2),
 (100, 3),
 (100, 4)]

In [42]:
# Hyperparameters tuning: the loop that iterates over all hyperparameter configurations (9 in total)
for i in range (0, 9):
    print(i+1)
    
    # Model specification:
    model = Doc2Vec(vector_size= a[i][0],
                alpha=0.01, # Learning rate at the start of the training
                min_alpha=0.0001, # Learning rate by the end of the training (last epoch)
                min_count=2, # Ignores words with frequencies lower than this
                window_size= a[i][1],
                dm =1, # Distributed memory model
                max_vocab_size=None,
                epochs = 100, # Train for 100 epochs
                seed=123) # Set random seed 

    # Build a vocabulary:
    model.build_vocab(tagged_data)
    
    # Train the model:
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    
    # Save the model:
    model.save("d2v.model")
    
    # Load the model: 
    model= Doc2Vec.load("d2v.model")
    
    # Store obtained document embeddings as a data frame:
    embeddings = pd.DataFrame(model.docvecs.vectors_docs)
    
    # Count how many times the model ranks the sentence as top 1 similar to itself and calculate the percentage out of the total number of sentences in the corpus:
    ranks = []
    second_ranks = []
    
    for doc_id in range(len(tagged_data)):
        inferred_vector = model.infer_vector(tagged_data[doc_id].words)
        sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
        rank = [docid for docid, sim in sims].index(str(doc_id))
        ranks.append(rank)
        
        second_ranks.append(sims[1])

    counter = collections.Counter(ranks)
    print(counter[0]/len(tagged_data)) 

1
0.9588004484304933
2
0.9613228699551569
3
0.9613228699551569
4
0.9624439461883408
5
0.9641255605381166
6
0.9630044843049327
7
0.9641255605381166
8
0.9638452914798207
9
0.9655269058295964


In [43]:
# The optimal model:
# Model specification:
model = Doc2Vec(vector_size=a[6][0],
                alpha=0.01, 
                min_alpha=0.0001,
                min_count=2, # Ignores words with frequencies lower than this
                window_size= a[6][1],
                dm =1,
                max_vocab_size=None,
                epochs = 100, seed=123)
    
# Build a vocabulary:
model.build_vocab(tagged_data)
   
# Train the model:
model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
model.save("d2v.model")
    
# Load the model: 
model= Doc2Vec.load("d2v.model")
    
# Display obtained document embeddings:
embeddings = pd.DataFrame(model.docvecs.vectors_docs)
    
# Save the data frame with all sentence embeddings into a csv file: to be used in the rest of the Text Summarization_Main.R script.
filename = r'~/Documents/R/Thesis/embeddings.csv'
embeddings.to_csv(filename, index = False)